!pip install grpcio

!pip install grpcio-tools

!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. image.proto

In [1]:
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. image.proto

In [8]:
from concurrent import futures
import time
import logging

import grpc
import cv2
import numpy as np

import image_pb2
import image_pb2_grpc

_ONE_DAY_IN_SECONDS = 60 * 60 * 24


class Rotater(image_pb2_grpc.RotateServicer):

    def rot90(self, request, context):
        img = cv2.imdecode(np.frombuffer(request.png,dtype=np.byte),cv2.IMREAD_COLOR)
        phot90 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

        r, buf = cv2.imencode(".png", phot90)
        return image_pb2.image(png = buf.tobytes())
    
    def raw_rot90(self, request, context):
        img = np.frombuffer(request.raw, dtype=np.uint8).reshape(request.rows, request.cols, 3)
        phot90 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        ret = image_pb2.raw_image()
        ret.raw = phot90.tobytes()
        ret.rows = phot90.shape[0]
        ret.cols = phot90.shape[1]
        return ret


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    image_pb2_grpc.add_RotateServicer_to_server(Rotater(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    try:
        while True:
            time.sleep(_ONE_DAY_IN_SECONDS)
    except KeyboardInterrupt:
        server.stop(0)

In [ ]:
logging.basicConfig()
serve()